In [109]:
import dedalus.public as d3
import numpy as np
import matplotlib.pyplot as plt
import scipy.constants as sc

'''
Code solving the 2D Laplacian equation:
    ∇²u = λu
with boundaries at x ∈ [0,1] and y ∈ [0,1], with homogeneous Dirichlet boundary conditions
and then plotting the first 10 eigenvalues calculated.
'''

#Bases
coords = d3.CartesianCoordinates('x', 'y')
dist = d3.Distributor(coords, dtype=np.float64)
xbasis = d3.Chebyshev(coords['x'], 64, bounds=(0,1), dealias=3/2)
ybasis = d3.Chebyshev(coords['y'], 64, bounds=(0,1), dealias=3/2)

#Fields
u = dist.Field(name='u', bases=(xbasis, ybasis))
lam = dist.Field(name='lam')
tau1 = dist.Field(name='tau1')
tau2 = dist.Field(name='tau2')
tau3 = dist.Field(name='tau3')
tau4 = dist.Field(name='tau4')


#Subs
x = dist.local_grid(xbasis)
y = dist.local_grid(ybasis)
lift_basis1 = xbasis.derivative_basis(1)
lift_basis2 = ybasis.derivative_basis(1) 
liftx = lambda A: d3.Lift(A, lift_basis1, -1)
lifty = lambda A: d3.Lift(A, lift_basis2, -1)

#Problem
problem = d3.EVP([u, tau1, tau2, tau3, tau4], eigenvalue=lam, namespace=locals())

problem.add_equation("lap(u) - lam*u + liftx(tau1) + liftx(tau2) + lifty(tau3) + lifty(tau4) = 0")
problem.add_equation("u(y = 0) = 0")
problem.add_equation("u(y = 1) = 0")
problem.add_equation("u(x = 0) = 0")
problem.add_equation("u(x = 1) = 0")

#Solver
solver = problem.build_solver()
solver.solve_dense(solver.subproblems[0])

ValueError: Non-square system: group=(None, None), I=4352, J=4100